# LLM Embedding

This notebook shows the embedding process of our full corpus that will be used later to retrieve documents relevant to given queries.

In [1]:
import warnings
import json
import torch

warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer

## Data Loading

In [2]:
file_path = '../data/corpus.json/clean_corpus_fr.json'

with open(file_path, 'r') as f:
        # Parse each line as an independent JSON object
        data = [json.loads(line) for line in f]
        
print(len(data), 'documents loaded.')

10676 documents loaded.


## Embedding

Start to vectorize the text by embedding them with the DistillBERT encoder.

In [3]:
# Create a list containing all text from the corpus
corpus = [document['text'] for document in data[:100]]
docids = [document['docid'] for document in data[:100]]

# Select the LLM used to perform the embeddings
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encode all the text
corpus_embeddings = model.encode(corpus, show_progress_bar=True)

# Display some information to the user
print("Information about the corpus embedding:\n"
      f"Number of document embedded: {len(corpus_embeddings)}\n"
      f"Vector size of each embedding: {corpus_embeddings[0].shape}")

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Information about the corpus embedding:
Number of document embedded: 100
Vector size of each embedding: (384,)


In [4]:
query = "Quelles sont les informations révélées par la Société norvégienne de radiodiffusion (NRK) concernant les adresses et les numéros des employés du complexe de l'abri antiatomique norvégien ?"  # wanted document: doc-fr-4878
top_k=10

query_embedding = model.encode(query)

# We use cosine-similarity and torch.topk to find the highest 5 scores
similarity_scores = model.similarity(query_embedding, corpus_embeddings)[0]
scores, indices = torch.topk(similarity_scores, k=top_k)

print(f"Query: {query}\n\n"
      f"{'#'*50}\n"
      f"Top {top_k} most similar sentences in corpus:\n"
      f"{'#'*50}\n")

for score, idx in zip(scores, indices):
        print(f"Docid: {docids[idx]}\n"
              f"Text:{corpus[idx][:50]}\n"
              f"(Score: {score:.4f})\n")

Query: Quelles sont les informations révélées par la Société norvégienne de radiodiffusion (NRK) concernant les adresses et les numéros des employés du complexe de l'abri antiatomique norvégien ?

##################################################
Top 10 most similar sentences in corpus:
##################################################

Docid: doc-fr-1029
Text:shōnen manga masahiro ikeno prépublié entre 2012 m
(Score: 0.3152)

Docid: doc-fr-4878
Text:continuité gouvernement cog principe établissement
(Score: 0.2756)

Docid: doc-fr-1342
Text:thumb 300px plan stations épuration skanstull erik
(Score: 0.2714)

Docid: doc-fr-6955
Text:effet indésirable médicament soin effet défavorabl
(Score: 0.2474)

Docid: doc-fr-1215
Text:centrale nucléaire golfech implantée commune golfe
(Score: 0.2404)

Docid: doc-fr-384
Text:thumb 350px right carte postale grande synagogue i
(Score: 0.2327)

Docid: doc-fr-4174
Text:bataille alger a opposé 1957 alger département alg
(Score: 0.2296)

Docid: doc-fr-41